# Toronto neighborhood

### Transform the data into a pandas dataframe

In [1]:
# import libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.request



In [2]:
# specify url

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

In [3]:
# parse the HTML from our URL into the BeautifulSoup parse tree format

soup = BeautifulSoup(page.text, "lxml")

In [11]:
#soup iterate

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

In [12]:
#convert list into Pandas DataFrame

torodf = pd.DataFrame(data = data,columns = columns)
torodf.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
# canadian english to usa english

torodf = torodf.rename(columns={'Neighbourhood':'Neighborhood'})

In [14]:
# droping NOT ASSIGNED

torodf = torodf[torodf.Borough != 'Not assigned']
torodf.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
# grouping dataframe

torodf["Neighborhood"] = torodf.groupby("Postal Code")["Neighborhood"].transform(lambda x: ', '.join(x))
torodf.head()


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
# drop duplicates

torodf = torodf.drop_duplicates()
torodf.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
# assign Borough to empty Neighbourhood

torodf['Neighborhood'].replace("Not assigned", torodf["Borough"],inplace=True)
torodf.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
# Reset index

torodf.reset_index(inplace=True)
torodf.head()

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
# Clean

torodf.drop(columns=['index'], inplace=True)
torodf.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
# shape

torodf.shape

(103, 3)

### Taking coordenates to built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name


In [21]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
# Built the new dataframe

torocoordf = pd.merge(left=torodf,right=df_data_1, left_on='Postal Code', right_on='Postal Code')
torocoordf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [23]:
torocoordf.shape

(103, 5)

### -/- Explore and cluster the neighborhoods in Toronto Borough -/-

In [24]:
# Import libraries 

import matplotlib.pyplot as plt 
%matplotlib inline 
from matplotlib import cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs


In [25]:
# API credentials

CLIENT_ID = 'QD2AGHVK4EGBFXSQ2SHYQPX0ARDPZ15BCELOFHFKKE03VCTJ' # your Foursquare ID
CLIENT_SECRET = 'SRHMFBI2R3JFND0FB15UDGD3SGVVXS1F3IIIHCJDPUZAYHAY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 60

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QD2AGHVK4EGBFXSQ2SHYQPX0ARDPZ15BCELOFHFKKE03VCTJ
CLIENT_SECRET:SRHMFBI2R3JFND0FB15UDGD3SGVVXS1F3IIIHCJDPUZAYHAY


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=torocoordf['Neighborhood'],
                                   latitudes=torocoordf['Latitude'],
                                   longitudes=torocoordf['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [29]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [32]:
# coordinates of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [33]:
# Create a map of Toronto

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(torocoordf['Latitude'], torocoordf['Longitude'], torocoordf['Borough'], torocoordf['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [34]:
# check the size of the toronto venues df

print(toronto_venues.shape)
toronto_venues.head()

(1816, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [35]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19


#### Analyze Each Neighborhood


In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# examine the new dataframe size

toronto_onehot.shape

(1816, 256)

In [38]:
#group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
1,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.017241,0.000000,0.000000,0.00,0.000000,0.0,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
8,"Business reply mail Processing Centre, South C...",0.052632,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000


In [51]:
# confirm size

toronto_grouped.shape

(94, 256)

In [39]:
#print each neighbourhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
num_top_venues = 5

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1               Skating Rink   0.2
2  Latin American Restaurant   0.2
3             Clothing Store   0.2
4             Breakfast Spot   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2     Coffee Shop  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0       Coffee Shop  0.10
1              Bank  0.10
2       Pizza Place  0.05
3  Sushi Restaurant  0.05
4          Pharmacy  0.05


----Bayview Village----
                        venue  freq
0          Chinese Restaurant  0.25
1                        Café  0.25
2                        Bank  0.25
3         Japanese Restaurant  0.25
4  Modern European Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.08
2      Sand

In [99]:
# making a pandas dataframe with that

# write function in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [100]:
# create the new dataframe and display the top 10 venues for each neighbourhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Clothing Store,Lounge,Breakfast Spot,Skating Rink,College Stadium,Colombian Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Skating Rink,Sandwich Place,Coffee Shop,Pharmacy,Gym,Pub,Curling Ice,Dance Studio,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Shopping Mall,Sandwich Place,Deli / Bodega,Bridal Shop,Restaurant,Ice Cream Shop,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Women's Store,Diner,Department Store,Dessert Shop,Dim Sum Restaurant,Distribution Center
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Juice Bar,Liquor Store,Locksmith,Indian Restaurant,Pub


#### Cluster Neighborhoods

In [101]:
# NaN values?

neighborhood_venues_sorted.columns[neighborhood_venues_sorted.isnull().any()]

Index([], dtype='object')

In [102]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [103]:
# create new dataframe

# add clustering labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torocoordf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,French Restaurant,Hockey Arena,Pizza Place,Portuguese Restaurant,Department Store,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Mexican Restaurant,Beer Store,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Coffee Shop,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Event Space,Boutique,Dance Studio,Deli / Bodega
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Diner,College Cafeteria,Yoga Studio,Bank,Bar,Smoothie Shop,Beer Bar,Sandwich Place,Distribution Center
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Coffee Shop,Restaurant,Japanese Restaurant,Beer Store,Clothing Store,Supermarket,Chinese Restaurant,Caribbean Restaurant,Café
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Fast Food Restaurant,Bank,Pet Store,Gastropub,Pharmacy,Gym / Fitness Center,Athletics & Sports,Intersection,Women's Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Café,Clothing Store,Pizza Place,Cosmetics Shop,Tea Room,Japanese Restaurant,Theater,Ramen Restaurant,Italian Restaurant


In [104]:
# Buscando valores NaN

toronto_merged.isnull().values.any()
toronto_merged.isnull().any()

Postal Code               False
Borough                   False
Neighborhood              False
Latitude                  False
Longitude                 False
Cluster Labels             True
1st Most Common Venue      True
2nd Most Common Venue      True
3rd Most Common Venue      True
4th Most Common Venue      True
5th Most Common Venue      True
6th Most Common Venue      True
7th Most Common Venue      True
8th Most Common Venue      True
9th Most Common Venue      True
10th Most Common Venue     True
dtype: bool

In [105]:
null_columns = toronto_merged.columns[toronto_merged.isnull().any()]
toronto_merged[null_columns].isnull().sum()
print(toronto_merged[toronto_merged.isnull().any(axis=1)][null_columns].head())

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
5              NaN                   NaN                   NaN   
11             NaN                   NaN                   NaN   
45             NaN                   NaN                   NaN   
52             NaN                   NaN                   NaN   
95             NaN                   NaN                   NaN   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
5                    NaN                   NaN                   NaN   
11                   NaN                   NaN                   NaN   
45                   NaN                   NaN                   NaN   
52                   NaN                   NaN                   NaN   
95                   NaN                   NaN                   NaN   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
5                    NaN                   NaN                   NaN   
11                   NaN  

In [106]:
toronto_merged = toronto_merged.dropna(how='any')
#toronto_merged.reset_index(inplace=True)

toronto_merged.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,French Restaurant,Hockey Arena,Pizza Place,Portuguese Restaurant,Department Store,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Mexican Restaurant,Beer Store,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Coffee Shop,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Event Space,Boutique,Dance Studio,Deli / Bodega
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Diner,College Cafeteria,Yoga Studio,Bank,Bar,Smoothie Shop,Beer Bar,Sandwich Place,Distribution Center
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Coffee Shop,Restaurant,Japanese Restaurant,Beer Store,Clothing Store,Supermarket,Chinese Restaurant,Caribbean Restaurant,Café
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Fast Food Restaurant,Bank,Pet Store,Gastropub,Pharmacy,Gym / Fitness Center,Athletics & Sports,Intersection,Women's Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Café,Clothing Store,Pizza Place,Cosmetics Shop,Tea Room,Japanese Restaurant,Theater,Ramen Restaurant,Italian Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Asian Restaurant,Pizza Place,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop


In [108]:
#toronto_merged.drop(columns=['index'], inplace=True)
#toronto_merged.head()

In [113]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged['Cluster Labels'].dtype

dtype('int64')

In [114]:
# create cluters map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

#### Cluster 1 - Pizza and sandwich

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,0,Pizza Place,Gym,Women's Store,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store
56,York,0,Coffee Shop,Discount Store,Sandwich Place,Convenience Store,Curling Ice,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
70,Etobicoke,0,Pizza Place,Coffee Shop,Intersection,Sandwich Place,Chinese Restaurant,Middle Eastern Restaurant,Discount Store,Diner,Dim Sum Restaurant,Cuban Restaurant
77,Etobicoke,0,Sandwich Place,Mobile Phone Shop,Pizza Place,Bus Line,Women's Store,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dim Sum Restaurant
93,Etobicoke,0,Pizza Place,Skating Rink,Sandwich Place,Coffee Shop,Pharmacy,Gym,Pub,Curling Ice,Dance Studio,Deli / Bodega


#### Cluster 2 - Coffee and more

In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Coffee Shop,French Restaurant,Hockey Arena,Pizza Place,Portuguese Restaurant,Department Store,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega
2,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Mexican Restaurant,Beer Store,Shoe Store
3,North York,1,Furniture / Home Store,Clothing Store,Coffee Shop,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Event Space,Boutique,Dance Studio,Deli / Bodega
4,Downtown Toronto,1,Coffee Shop,Diner,College Cafeteria,Yoga Studio,Bank,Bar,Smoothie Shop,Beer Bar,Sandwich Place,Distribution Center
7,North York,1,Gym,Coffee Shop,Restaurant,Japanese Restaurant,Beer Store,Clothing Store,Supermarket,Chinese Restaurant,Caribbean Restaurant,Café
8,East York,1,Pizza Place,Fast Food Restaurant,Bank,Pet Store,Gastropub,Pharmacy,Gym / Fitness Center,Athletics & Sports,Intersection,Women's Store
9,Downtown Toronto,1,Coffee Shop,Café,Clothing Store,Pizza Place,Cosmetics Shop,Tea Room,Japanese Restaurant,Theater,Ramen Restaurant,Italian Restaurant
10,North York,1,Asian Restaurant,Pizza Place,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
12,Scarborough,1,Bar,Home Service,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Curling Ice
13,North York,1,Gym,Coffee Shop,Restaurant,Japanese Restaurant,Beer Store,Clothing Store,Supermarket,Chinese Restaurant,Caribbean Restaurant,Café


#### Cluster 3 - The Parking cluster

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Food & Drink Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
21,York,2,Women's Store,Convenience Store,Pool,Park,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Cupcake Shop
32,Scarborough,2,Playground,Women's Store,Cupcake Shop,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
35,East York,2,Park,Convenience Store,Cupcake Shop,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
64,York,2,Park,Convenience Store,Cupcake Shop,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
66,North York,2,Park,Convenience Store,Construction & Landscaping,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Cupcake Shop
68,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Cuban Restaurant
83,Central Toronto,2,Park,Playground,Trail,Lawyer,College Cafeteria,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store
85,Scarborough,2,Park,Playground,Dim Sum Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Women's Store
91,Downtown Toronto,2,Park,Playground,Trail,Dog Run,Distribution Center,Discount Store,Diner,Doner Restaurant,Creperie,Dim Sum Restaurant


#### Cluster 4 - Fast food and drugstore

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 5 - Sports and more

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Baseball Field,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
101,Etobicoke,4,Baseball Field,Business Service,Women's Store,Curling Ice,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
